In [8]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import os
import glob
import json
from pprint import pprint
import collections
from operator import itemgetter, attrgetter, methodcaller
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import string
%matplotlib inline

## getting list of file names

In [2]:
# get a list of files
list_of_files = []

for root, dirs, files in os.walk("lastfm_train/"):
    for file in files:
        if file.endswith(".json"):
            list_of_files.append(os.path.join(root, file)) 

## getting songs id with tags and match the name and artist with each song in the four tag files

In [3]:
df_happy = pd.read_csv("happy_counts.csv")
df_relax = pd.read_csv("relax_counts.csv")
df_energetic = pd.read_csv("energetic_counts.csv")
df_sad = pd.read_csv("sad_counts.csv")

happy_id = df_happy['track_id'].values
sad_id = df_sad['track_id'].values
energetic_id = df_energetic['track_id'].values
relax_id = df_relax['track_id'].values

In [4]:
#gets the track id, title, and artist name from last.fm dataset for all the songs
song_info_list = []
i = 0
for fileName in list_of_files:
    if i % 20000 == 0:
        print i,
    with open(fileName) as json_data:
        d = json.loads(json_data.read())
        json_data.close()
        song_info_list.append([d[u'track_id'].encode('ascii','ignore'),d[u'title'].encode('ascii','ignore'),d[u'artist'].encode('ascii','ignore')])
    i+=1
#append tags to the songs
index = 0
for item in song_info_list:
    i, j, k = item
    tag = ''
    if index % 20000 == 0:
        print index,
    if i in happy_id:
        tag = 'happy'
    elif i in sad_id:
        tag = 'sad'
    elif i in relax_id:
        tag = 'relax'
    elif i in energetic_id:
        tag = 'energetic'
    item.append(tag)
    index += 1

0 20000 40000 60000 80000 100000 120000 140000 160000 180000 200000 220000 240000 260000 280000 300000 320000 340000 360000 380000 400000 420000 440000 460000 480000 500000 520000 540000 560000 580000 600000 620000 640000 660000 680000 700000 720000 740000 760000 780000 800000 820000 0 20000 40000 60000 80000 100000 120000 140000 160000 180000 200000 220000 240000 260000 280000 300000 320000 340000 360000 380000 400000 420000 440000 460000 480000 500000 520000 540000 560000 580000 600000 620000 640000 660000 680000 700000 720000 740000 760000 780000 800000 820000


In [5]:
#save to a data frame
labels = ['id', 'Name', 'Aritist','tag']
df_with_tag = pd.DataFrame.from_records(song_info_list, columns=labels)
df_with_tag.head()

,id,Name,Aritist,tag
0,TRAAAAK128F9318786,Scream,Adelitas Way,
1,TRAAAAV128F421A322,A Poor Recipe For Civic Cohesion,Western Addiction,
2,TRAAAAW128F429D538,I Didn't Mean To,Casual,
3,TRAAABD128F429CF47,Soul Deep,The Box Tops,
4,TRAAACV128F423E09E,Come On (Album Version),Super Deluxe,


In [6]:
df_new = df_with_tag[df_with_tag['tag']!= '']
df_new.to_csv("song_info_with_tag.csv")

## generate the name of songs and artists in lowercase for matching the song lyrics 

In [9]:
song_name = df_new['Name'].values
song_artist = df_new['Aritist'].values
df_new.head()

,id,Name,Aritist,tag
8,TRAAAED128E0783FAB,It's About Time,Jamie Cullum,relax
43,TRAAAZF12903CCCF6B,Break My Stride,Matthew Wilder,happy
56,TRAABIG128F9356C56,Walk the Walk,Poe,energetic
101,TRAACER128F4290F96,Setting Fire to Sleeping Giants,The Dillinger Escape Plan,energetic
144,TRAACZN128F93236B1,Seaweed,Saul Williams,relax


In [11]:
def scrape(s,a):
    exclude = set(string.punctuation)
    s = s.lower().replace(" ", "")
    a = a.lower().replace(" ", "")
    if a.startswith("the"):
        a = a.replace("the", "")
    return ''.join(ch for ch in s if ch not in exclude),''.join(ch for ch in a if ch not in exclude)

In [12]:
scrape_song = [scrape(song_name[i],song_artist[i]) for i in range(len(song_name))]
scrape_song[:5]

[('itsabouttime', 'jamiecullum'),
 ('breakmystride', 'matthewwilder'),
 ('walkthewalk', 'poe'),
 ('settingfiretosleepinggiants', 'dillingerescapeplan'),
 ('seaweed', 'saulwilliams')]

In [13]:
labels = ['Name', 'Aritist']
df_2 = pd.DataFrame.from_records(scrape_song, columns=labels)
df_2.to_csv("Song_Scraping.csv")

In [15]:
df_new = df_new.reset_index()
df_new.head()

,index,id,Name,Aritist,tag
0,8,TRAAAED128E0783FAB,It's About Time,Jamie Cullum,relax
1,43,TRAAAZF12903CCCF6B,Break My Stride,Matthew Wilder,happy
2,56,TRAABIG128F9356C56,Walk the Walk,Poe,energetic
3,101,TRAACER128F4290F96,Setting Fire to Sleeping Giants,The Dillinger Escape Plan,energetic
4,144,TRAACZN128F93236B1,Seaweed,Saul Williams,relax


## matching the name artist with lyrcis local location

In [16]:
def match(song, artist, string):
    s = song.split()
    st = string.split('_')
    l = [1. if i in st[1] else 0. for i in s]
    return np.mean(l) >=0.5 and artist in st[0] 

In [18]:
song_lyrics = []
j = 1
for i in range(len(song_name)):
    lyrics = ''
    song,artist = song_name[i], song_artist[i]
#     if artist.upper().startswith('A') or artist.upper().startswith('T') or artist.upper().startswith('M'):
#         for path, subdirs, files in os.walk('./Lyrics/T,M,A/'):
#             for name in files:
#                 if name.endswith('.txt') and match(song,artist,name):
#                     lyrics = os.path.join(path, name)
#                     j+=1
#                     #time.sleep(1)
#     elif artist.upper().startswith('D') or artist.upper().startswith('E') or artist.upper().startswith('F') or artist.upper().startswith('G') or artist.upper().startswith('H') or artist.upper().startswith('I') or artist.upper().startswith('J') or artist.upper().startswith('K') or artist.upper().startswith('L'):
#         for path, subdirs, files in os.walk('./Lyrics/lyrics_D_to_L/'):
#             for name in files:
#                 if name.endswith('.txt') and match(song,artist,name): #and (not name.startswith('mxm') or not name.startswith('user')):
#                     lyrics = os.path.join(path, name)
#                     j+=1
#     elif artist.upper().startswith('N') or artist.upper().startswith('O') or artist.upper().startswith('P'):
#         for path, subdirs, files in os.walk('./Lyrics/lyrics_N_to_P/'):
#             for name in files:
#                 if name.endswith('.txt') and match(song,artist,name): #and (not name.startswith('mxm') or not name.startswith('user')):
#                     lyrics = os.path.join(path, name)
#                     j+=1
    if lyrics == '':
        for path, subdirs, files in os.walk('./lyrics_new/'):
            for name in files:
                if name.endswith('.txt') and list(scrape(song, artist))[0] in name and list(scrape(song, artist))[1] in name: #and (not name.startswith('mxm') or not name.startswith('user')):
                    lyrics = os.path.join(path, name)
                    j+=1
    song_lyrics.append(lyrics)
    if i % 1000 ==0:
        print "i = "+ str(i),
        print "j = "+ str(j),

 i = 0 j = 2 i = 1000 j = 651 i = 2000 j = 1202 i = 3000 j = 1659 i = 4000 j = 2240 i = 5000 j = 2847 i = 6000 j = 3376 i = 7000 j = 3827 i = 8000 j = 4359 i = 9000 j = 4676 i = 10000 j = 5079 i = 11000 j = 5690 i = 12000 j = 5752 i = 13000 j = 5756 i = 14000 j = 5761 i = 15000 j = 5882


In [19]:
song_id = df_new['id'].values
song_tag = df_new['tag'].values
z = zip(song_id,song_name,song_artist,song_tag,song_lyrics)

In [21]:
labels = ['id', 'Name', 'Aritist','tag','Song_txt_loc']
df_with_txt = pd.DataFrame.from_records(z, columns=labels)
df_with_txt.head()

,id,Name,Aritist,tag,Song_txt_loc
0,TRAAAED128E0783FAB,It's About Time,Jamie Cullum,relax,./lyrics_new/jamiecullum_itsabouttime.txt
1,TRAAAZF12903CCCF6B,Break My Stride,Matthew Wilder,happy,
2,TRAABIG128F9356C56,Walk the Walk,Poe,energetic,
3,TRAACER128F4290F96,Setting Fire to Sleeping Giants,The Dillinger Escape Plan,energetic,./lyrics_new/dillingerescapeplan_settingfireto...
4,TRAACZN128F93236B1,Seaweed,Saul Williams,relax,


In [23]:
new_df_with_txt = df_with_txt[df_with_txt['Song_txt_loc'] != '']
new_df_with_txt.to_csv("song_with_txt_loc_new.csv")
new_df_with_txt = new_df_with_txt.reset_index()
new_df_with_txt.head()

,index,id,Name,Aritist,tag,Song_txt_loc
0,0,TRAAAED128E0783FAB,It's About Time,Jamie Cullum,relax,./lyrics_new/jamiecullum_itsabouttime.txt
1,3,TRAACER128F4290F96,Setting Fire to Sleeping Giants,The Dillinger Escape Plan,energetic,./lyrics_new/dillingerescapeplan_settingfireto...
2,7,TRAADVO128E07999E9,Oh God,Jamie Cullum,relax,./lyrics_new/jamiecullum_ohgod.txt
3,8,TRAAFGQ128F427D884,One Last Time,The Kooks,sad,./lyrics_new/kooks_onelasttime.txt
4,10,TRAAGCZ128F93210FD,Let's Get It Started,Black Eyed Peas,happy,./lyrics_new/blackeyedpeas_letsgetitstarted.txt


### function to tokenize lyrics to string form without wired symbols

In [24]:
import nltk
from nltk import word_tokenize
def LyricsToSentences(lyrics):
    if '\r' in lyrics:
        lines = lyrics.split('\r')
        for i, l in enumerate(lines):
            if "<i>" in l:
                lines[i] = " "
        lines = ' '.join(lines).replace("\n \n", "\n").split(' \n')
        l = ' '
        for i in lines:
            l+= i
        return l
    elif "\n\n" in lyrics:
        lines = lyrics.split('\n')
        for i, l in enumerate(lines):
            if "<i>" in l:
                lines[i] = " "
        lines = ' '.join(lines).split('\r')
        l = ' '
        for i in lines:
            l+= i
        return l

In [25]:
df_with_txt_val = new_df_with_txt.values.tolist()
for i in range(len(df_with_txt_val)):
    file_name = df_with_txt_val[i][5]
    with open(file_name, 'r') as f:
        fa = f.read()#.encode('ascii','ignore')
        df_with_txt_val[i].append(LyricsToSentences(fa))

In [26]:
labels = ['index','id', 'Name', 'Aritist','tag','Song_txt_loc','Song_lyrics']
with_txt_val2 = pd.DataFrame.from_records(df_with_txt_val, columns=labels)
with_txt_val2.to_csv("Song_with_lyrics_new.csv")

In [27]:
pd.read_csv("Song_with_lyrics_new.csv").head()

,Unnamed: 0,index,id,Name,Aritist,tag,Song_txt_loc,Song_lyrics
0,0,0,TRAAAED128E0783FAB,It's About Time,Jamie Cullum,relax,./lyrics_new/jamiecullum_itsabouttime.txt,Walking down to the water's edgeWhere I have ...
1,1,3,TRAACER128F4290F96,Setting Fire to Sleeping Giants,The Dillinger Escape Plan,energetic,./lyrics_new/dillingerescapeplan_settingfireto...,First off Let me say you look so tired... Res...
2,2,7,TRAADVO128E07999E9,Oh God,Jamie Cullum,relax,./lyrics_new/jamiecullum_ohgod.txt,I know it's been a while since I have talked ...
3,3,8,TRAAFGQ128F427D884,One Last Time,The Kooks,sad,./lyrics_new/kooks_onelasttime.txt,Can I hold you one last time To fight the fee...
4,4,10,TRAAGCZ128F93210FD,Let's Get It Started,Black Eyed Peas,happy,./lyrics_new/blackeyedpeas_letsgetitstarted.txt,Let's get it started in here... And the bass...
